<a href="https://colab.research.google.com/github/MesutCinar/fvtt_dsa_json2pdf/blob/main/FoundryChar2PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modul zur Konvertierung von HTML nach PDF installieren

In [6]:
!pip install xhtml2pdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.0/447.0 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.5 MB/s eta 0:00:00
  Created wheel for svglib: filename=svglib-1.5.1-py3-none-any.whl size=30906 sha256=6717137b812d8adac7b159dc0aadb70203867d4f21f261a1b8581a73b3f10709
  Stored 

JSON Charakter Datei laden

In [156]:
import json
import re
import textwrap
from xhtml2pdf import pisa

TAG_RE = re.compile(r'<[^>]+>')

json_file="/content/drive/MyDrive/Colab Notebooks/fvtt-Actor-hesindian-schlangentreu-pCXAwjkUPBP5X2uz.json"

with open(json_file, 'r') as f:
    data = json.load(f)

print(data)

{'name': 'Hesindian Schlangentreu', 'img': 'modules/dsa5-drachenritter/icons/characters/avatar/HesindianSchlangentreu.webp', 'items': [{'_id': 'OCRi6UuKBIHCbZuF', 'name': 'Dukaten', 'type': 'money', 'flags': {'core': {'sourceId': 'Item.BGfRbsrZ2M2Y2wZ3'}}, 'img': 'systems/dsa5/icons/money-D.webp', 'effects': [], 'system': {'description': {'value': ''}, 'gmdescription': {'value': ''}, 'price': {'value': 10}, 'quantity': {'value': 21}, 'weight': {'value': 0}, 'effect': {'value': '', 'attributes': ''}, 'parent_id': 0, 'subtype': 0}, 'ownership': {'default': 0, 'GTlurwFoZkD2n9wE': 3}, 'folder': None, 'sort': 0, '_stats': {'systemId': 'dsa5', 'systemVersion': '6.0.12', 'coreVersion': '12.330', 'createdTime': None, 'modifiedTime': 1725137086516, 'lastModifiedBy': 'PiM8c83PTw5umVYe', 'compendiumSource': 'Item.BGfRbsrZ2M2Y2wZ3', 'duplicateSource': None}}, {'_id': 'xN0OtnyZqB4BaWAX', 'name': 'Silber', 'type': 'money', 'flags': {'core': {'sourceId': 'Item.ntX4yPa0e9rmmqSZ'}}, 'img': 'systems/dsa

In [210]:
class DSAChar():
  name = None
  details = None
  experience_total = None
  experience_spent = None
  characteristics = None
  char_items = None

  def __init__(self, name):
    self.name = name
    self.details = {}
    self.experience_total = 0
    self.experience_spent = 0
    self.characteristics = {}
    self.char_items = {}


In [232]:
dsa_char = DSAChar(data["name"])

dsa_char.experience_total = data["system"]["details"]["experience"]["total"]
dsa_char.experience_spent = data["system"]["details"]["experience"]["spent"]

for key,value in data["system"]["details"].items():
    if "value" in value:
        dsa_char.details[key] = '<br/>\n'.join(textwrap.wrap(TAG_RE.sub('', value["value"]), width=80))
        #dsa_char.details[key] = textwrap.shorten(TAG_RE.sub('', value["value"]), width=40)
        if len(dsa_char.details[key]) < 1:
            dsa_char.details[key] = "n/a"

for key,value in data["system"]["characteristics"].items():
    dsa_char.characteristics[value["label"]] = value["initial"] + value["advances"]

for item in data["items"]:
    item_name = item["name"]
    item_type = item["type"]
    if item_type not in dsa_char.char_items:
        dsa_char.char_items[item_type] = {}

    if "quantity" in item["system"]:
        dsa_char.char_items[item_type][item_name] = item["system"]["quantity"]["value"]
    elif "talentValue" in item["system"]:
        dsa_char.char_items[item_type][item_name] = item["system"]["talentValue"]["value"]
    elif "maxRank" in item["system"]:
        dsa_char.char_items[item_type][item_name] = item["system"]["maxRank"]["value"]
    elif "APValue" in item["system"]:
        dsa_char.char_items[item_type][item_name] = item["system"]["APValue"]["value"]
    else:
        print("Fehlt: " + item["name"])

print(dsa_char.name)
print(dsa_char.details)
print(dsa_char.characteristics)
print(dsa_char.char_items)

Hesindian Schlangentreu
{'species': 'Mensch', 'gender': 'n/a', 'culture': 'Mittelreich', 'career': 'Magier', 'socialstate': 'n/a', 'home': 'n/a', 'family': 'n/a', 'age': 'n/a', 'haircolor': 'n/a', 'eyecolor': 'n/a', 'height': 'n/a', 'weight': 'n/a', 'distinguishingmark': 'n/a', 'biography': 'Hesindian Schlangentreu wurde in einem Magierturm von einem persönlichen<br/>\nLehrmeister zum Magier ausgebildet. Neben dem Erlernen der magischen Künste und<br/>\nder Alchimie, beinhaltete diese Ausbildung aber auch die Unterstützung seines<br/>\nLehrmeisters bei dessen Forschungen und die unliebsame Erfüllung eher lästiger<br/>\nPflichten im Haushalt. Körperliche Ertüchtigungen gehörten abgesehen von ein<br/>\npaar Unterrichtsstunden im Stockfechten mit dem Magierstab eher nicht dazu.Schon<br/>\nimmer neugierig auf die Welt außerhalb des Turmes, hielt es Hesindian nach dem<br/>\nEnde seiner Ausbildungszeit nicht lange bei seinem Lehrmeister aus. Doch junge<br/>\nMagier haben es gar nicht so einf

HTML Darstellung erzeugen

In [239]:
html_content = '''<!DOCTYPE html>
        <html>
        <head>
            <title>Sample Company - Empfangsbestätigung</title>
            <style>
               table {border-collapse: collapse;}
               .td_left {width: 200px; background-color: #eeeeee; padding: 2px; border: 1px solid; text-align: right;}
               .td_right {background-color: #ffffff; padding: 2px; border: 1px solid;}
            </style>
        </head>
        <body>'''

html_content = html_content + '''<h1>{}</h1>'''.format(dsa_char.name)


In [240]:
html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Erfahrungspunkte")
html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
html_content = html_content + '''<tr>'''
html_content = html_content + '''<td class=\"td_left\">Total - verteilt = übrig</td><td class=\"td_right\">{} - {} = <b>{}</b></td>'''.format(dsa_char.experience_total, dsa_char.experience_spent, dsa_char.experience_total - dsa_char.experience_spent)
html_content = html_content + '''</tr>\n'''
html_content = html_content + '''</table>\n'''


html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Details")
html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
for key, value in dsa_char.details.items():
    html_content = html_content + '''<tr>'''
    html_content = html_content + '''<td class=\"td_left\">{}</td><td class=\"td_right\">{}</td>'''.format(key,value)
    html_content = html_content + '''</tr>\n'''
html_content = html_content + '''</table>\n'''


html_content = html_content + '''\n<h2>{}</h2>\n'''.format("Eigenschaften")
html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
for characteristic_key, characteristic_value in dsa_char.characteristics.items():
    html_content = html_content + '''<tr>'''
    html_content = html_content + '''<td class=\"td_left\">{}</td><td class=\"td_right\">{}</td>'''.format(characteristic_key,characteristic_value)
    html_content = html_content + '''</tr>\n'''
html_content = html_content + '''</table>\n'''

for item_type in dsa_char.char_items:
    if len(dsa_char.char_items[item_type]) < 1:
      continue

    html_content = html_content + '''\n<h2>{}</h2>\n'''.format(item_type)

    html_content = html_content + '''<table border="1" style="border: 1px solid;">\n'''
    for key,value in dsa_char.char_items[item_type].items():
        html_content = html_content + '''<tr>'''
        html_content = html_content + '''<td class=\"td_left\">{}</td><td class=\"td_right\">{}</td>'''.format(key,value)
        html_content = html_content + '''</tr>\n'''

    html_content = html_content + '''</table>\n'''


In [241]:
html_content = html_content + '''</body></html>'''

print(html_content)

<!DOCTYPE html>
        <html>
        <head>
            <title>Sample Company - Empfangsbestätigung</title>
            <style>
               table {border-collapse: collapse;}
               .td_left {width: 200px; background-color: #eeeeee; padding: 2px; border: 1px solid; text-align: right;}
               .td_right {background-color: #ffffff; padding: 2px; border: 1px solid;}
            </style>
        </head>
        <body><h1>Hesindian Schlangentreu</h1>
<h2>Erfahrungspunkte</h2>
<table border="1" style="border: 1px solid;">
<tr><td class="td_left">Total - verteilt = übrig</td><td class="td_right">1170 - 1161 = <b>9</b></td></tr>
</table>

<h2>Details</h2>
<table border="1" style="border: 1px solid;">
<tr><td class="td_left">species</td><td class="td_right">Mensch</td></tr>
<tr><td class="td_left">gender</td><td class="td_right">n/a</td></tr>
<tr><td class="td_left">culture</td><td class="td_right">Mittelreich</td></tr>
<tr><td class="td_left">career</td><td class="td_right"

HTML nach PDF konvertieren

In [242]:
# Convert the generated HTML to PDF
filename = "/content/drive/MyDrive/Colab Notebooks/" + dsa_char.name + ".pdf"
with open(filename, "wb") as pdf_file:
    pisa_status = pisa.CreatePDF(html_content, dest=pdf_file)
    print(pisa_status)
